Spectral Clustering Example.

I followed the example from https://www.planet.com/docs/api-quickstart-examples/step-2-download/ to download a geotif file. It's been saved as `redding.tiff` in the current directory.

In addition to `dask-ml`, we'll use `rasterio` to read the data and `matplotlib` to plot the figures.
I'm just working on my laptop, so we could use either the threaded or distributed scheduler. I'll use the distributed scheduler for the diagnostics.

In [ ]:
%matplotlib inline

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import dask.array as da
from dask_ml.cluster import SpectralClustering
from dask.distributed import Client

In [ ]:
with rasterio.open('redding.tiff') as dataset:
    arr = dataset.read(1)

arr = arr.astype(float)
# Rescale for the clustering algorithm
arr = (arr - arr.mean()) / arr.std()
# Subsample to the upper-left quadrant
arr = arr[:2500, :2500]
plt.imshow(arr);

In [ ]:
client = Client(processes=False)
client

We'll reshape the image to be how dask-ml / scikit-learn expect it: `(n_samples, n_features)` where n_features is 1 in this case. Then we'll persist that in memory. We still have a small dataset at this point. The large dataset, which dask helps us manage, is the intermediate `n_samples x n_samples` array that spectral clustering operates on. For our 2,500 x 2,500 pixel subset, that's ~50

In [ ]:
X = da.from_array(arr.reshape(-1, 1), chunks=100_000)
X = client.persist(X)

And we'll fit the estimator.

In [ ]:
clf = SpectralClustering(n_clusters=4, random_state=0,
                         kmeans_params={'init_max_iter': 5})

In [ ]:
%time clf.fit(X)

In [ ]:
labels = clf.assign_labels_.labels_.compute()

c = labels.reshape(arr.shape)

fig, axes = plt.subplots(ncols=2, figsize=(12, 6))
axes[0].imshow(arr)
axes[1].imshow(c);

axes[0].set_title("Image")
axes[1].set_title("Clustered")

for ax in axes:
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)